# Imports

In [ ]:
from __future__ import annotations
import itertools
import joblib
import warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, PowerTransformer
from skopt.searchcv import BayesSearchCV

warnings.simplefilter(action="ignore", category=UserWarning)

%matplotlib inline
pd.options.display.max_columns = None
plt.style.use("ggplot")

In [ ]:
df: pd.DataFrame = pd.read_csv('wine-quality.csv')

# Data Preparation

## Check for duplicate rows

In [ ]:
df.shape

In [ ]:
df.duplicated().sum()

In [ ]:
df = df.drop_duplicates()
df.shape

## Change spaces in column names to underscores

In [ ]:
df.columns = df.columns.str.replace(' ', '_')


## Check for missing values

In [ ]:
df.isna().sum()

# Exploratory Data Analysis

## Basic information about the dataset

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.describe(include="object")

In [ ]:
numeric_features: list[str] = list(df.select_dtypes(exclude="object").drop(columns=["quality"]).columns)
categorical_features: list[str] = list(df.select_dtypes(include="object").columns)

<hr />

## Distribution of each feature

In [ ]:
fig, axes = plt.subplots(4, 3, figsize=(20, 16), tight_layout=True)

for i, ax in enumerate(axes.flat):
    if i == 11: break
    sns.kdeplot(df[numeric_features[i]], fill=True, ax=ax)

These histograms tell me that the distribution of the data for all of the skewed to the left. This means that I will need to do some data transformation on the fields I decide to keep, so that the data is more normally distributed.

<hr />

In [ ]:
fig, axes = plt.subplots(11, 1, figsize=(20, 16.5), tight_layout=True)

for i, ax in enumerate(axes.flat):
    sns.boxplot(x=df[numeric_features[i]], ax=ax)

These boxplots tell me that there are some outliers in the data. I will need to decide if I want to keep these outliers or not.

<hr />

## Handling Outliers

For each of the fields, I decided to do some trail and error to find out what percentile of the data (top and bottom percentile) I want to replace with the median value of the column, keeping the percentile value change to a minimum so as to not impute too many fields, but also to remove as many outliers as possible. Here are the outcomes of my trail and erroring

In [ ]:
trim_config: dict[str, tuple[int, int]] = {
    "fixed_acidity": (0.03, 0.92),
    "volatile_acidity": (0, 0.81),
    "citric_acid": (0.06, 0.95),
    "residual_sugar": (0, 0.93),
    "chlorides": (0, 0.92),
    "free_sulfur_dioxide": (0, 0.9),
    "total_sulfur_dioxide": (0, 0.95),
    "density": (0, 0.99),
    "pH": (0.01, 0.98),
    "sulphates": (0, 0.96),
    "alcohol": (0, 0.99),
    "quality": (0.01, 0.97)
}

for feature, iqr_range in trim_config.items():
    df[f"{feature}_trimmed"] = df[feature].clip(
        df[feature].quantile(iqr_range[0]),
        df[feature].quantile(iqr_range[1])
    )

In [ ]:
fig, axes = plt.subplots(12, 1, figsize=(20, 24), tight_layout=True)

for i, ax in enumerate(axes.flat):
    if i == 11:
        feature = "quality"
    else:
        feature = numeric_features[i]

    sns.boxplot(
        pd.melt(df[[feature, f"{feature}_trimmed"]]),
        x="value",
        y="variable",
        ax=ax
    ).set(
        xlabel=None,
        ylabel=None
    )

### Overwriting the original columns with the trimmed columns

In [ ]:
for feature in numeric_features + ["quality"]:
    df[feature] = df[f"{feature}_trimmed"]
    df: pd.DataFrame = df.drop(columns=[f"{feature}_trimmed"])

## Correlation of each feature

In [ ]:
fig, ax = plt.subplots(figsize=(16, 8))

sns.heatmap(
    df[numeric_features + ["quality"]].corr(),
    vmin=-1,
    vmax=1,
    annot=True,
    cmap="coolwarm",
    ax=ax,
)

This correlation heatmap tells me that the features have a moderate correlation with each other. This means that I will need to do some feature selection to reduce the number of features I use in my model.

<hr />

## Correlation of each feature with the target

In [ ]:
fig, axes = plt.subplots(4, 3, figsize=(20, 16), tight_layout=True)

for i, ax in enumerate(axes.flat):
    if i == 11: break
    sns.lineplot(df, x="quality", y=numeric_features[i], ax=ax)

These graphs tell me that some of the features are more likely to affect the quality of the wine than others since the lines are more linear than the others, and can be used to predict the quality of the wine better. 

<hr />

# Feature Engineering & Model Building

## Normally Distributing the Data

I decided to transform each column individually so that they aren't skewed to either direction. I will be applying these changes to the data frame directly but rather by doing it in the pipeline

In [ ]:
boxcox_transformer = PowerTransformer(method="box-cox")

Here I am plotting the before and after boxcox transformation for each numeric feature

In [ ]:
fig, axes = plt.subplots(8, 3, figsize=(20, 24), tight_layout=True)

for i, ax in enumerate(axes.flat):
    if i in [20, 23]: continue

    # some magic to get the right feature name
    feature = numeric_features[i - ((i % 6 > 2) + i // 6) * 3]
    if i % 6 > 2:
        data = PowerTransformer(method="box-cox").fit_transform(df[[feature]])
        data.shape = (data.shape[0],)
        sns.kdeplot(data, fill=True, color="green", ax=ax)
    else:
        sns.kdeplot(df[feature], fill=True, ax=ax)

## Normalizing the data

I am using a pipeline with a standard scaler to normalize my numeric columns

In [ ]:
normalizing_transformer = MinMaxScaler()

## One Hot Encoding

I am using a pipeline with a one hot encoder to one hot encode the categorical features

In [ ]:
onehot_transformer = OneHotEncoder(handle_unknown="ignore")

## Classifying the target

I decided to split the output into 3 classes, 1 for low quality, 1 for medium quality, and 1 for high quality. This is because my output variable is not exactly continuous (since the value is either 3, 4, 5, 6, 7, 8 or 9). However if I use regression to solve this, I will have to use a continuous output variable, so I decided to use classification instead.

In [ ]:
sns.histplot(df["quality"], bins=10)

From the above histogram we can roughly split the data into 3 classes, 1 for low quality, 1 for medium quality, and 1 for high quality. I will use this to classify the data.

In [ ]:
df["quality_class"] = np.where(df["quality"] <= 4, "low", np.where(df["quality"] >= 7, "high", "medium"))

In [ ]:
sns.histplot(df["quality_class"])

In [ ]:
df = df.drop(columns=["quality"])

## First Draft of Model

I need to have at least one draft of the model before I can decide which features to keep, remove and scale for Feature Engineering. Becuase of this, I will run a simple RandomForestClassifier model with BayesSearchCV to find the best parameters for the model.

In [ ]:
running_bayes = True

The parameters here were found through a previous search, in the cases where I don't want to run bayes again since it can be very time consuming during testing

In [ ]:
bayes_optimal = {
    "max_depth": 7854,
    "min_samples_leaf": 1,
    "min_samples_split": 5,
    "n_estimators": 179
}

In [ ]:
rfc_pipeline = Pipeline([
    ("transformer", ColumnTransformer([
        ("numeric", Pipeline([
            ("boxcox", boxcox_transformer),
            ("normalizer", normalizing_transformer),
        ]), df[numeric_features].columns),
        ("categorical", Pipeline([
            ("onehot", onehot_transformer)
        ]), df[categorical_features].columns)
    ])),
    ("classifier", RandomForestClassifier(
        **({} if running_bayes else bayes_optimal),
        random_state=hash("2100326D") % 2 ** 32,
        n_jobs=-1
    ))
])

In [ ]:
X = df.drop(columns=["quality_class"])
y = df["quality_class"]

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=hash("2100326D") % 2 ** 32)
X_train: pd.DataFrame
X_test: pd.DataFrame
y_train: pd.Series
y_test: pd.Series

Now depending on whether I've run BayesSearchCV, I will either load the best parameters found or calculate them again

In [ ]:
if running_bayes:
    # Fix a bug in scikit-optimize
    np.int = int

    bs = BayesSearchCV(
        rfc_pipeline,
        {
            "classifier__max_depth": (1, 10000),
            "classifier__min_samples_leaf": (1, 5),
            "classifier__min_samples_split": (2, 5),
            "classifier__n_estimators": (1, 1000)
        },
        n_iter=100,
        scoring="accuracy",
        cv=5,
        n_jobs=-1,
        verbose=2
    )

    bs.fit(X_train, y_train)
    bayes_optimal = {
        "max_depth": bs.best_params_["classifier__max_depth"],
        "min_samples_leaf": bs.best_params_["classifier__min_samples_leaf"],
        "min_samples_split": bs.best_params_["classifier__min_samples_split"],
        "n_estimators": bs.best_params_["classifier__n_estimators"]
    }

    print("Best params:", bayes_optimal)
    print("Accuracy Score: %.4f" % accuracy_score(y_test, bs.predict(X_test)))
else:
    rfc_pipeline.fit(X_train, y_train)

    print("Accuracy Score: %.4f" % accuracy_score(y_test, rfc_pipeline.predict(X_test)))

In [ ]:
fig, ax = plt.subplots(figsize=(16, 8))

rfc_pipeline: Pipeline = bs.best_estimator_ if running_bayes else rfc_pipeline
_classifier: RandomForestClassifier = rfc_pipeline.named_steps.classifier
_onehot: OneHotEncoder = rfc_pipeline.named_steps.transformer.named_transformers_.categorical.named_steps.onehot

importances = pd.DataFrame(
    _classifier.feature_importances_,
    index=list(X_train.drop(columns=["type"]).columns) + list(_onehot.get_feature_names_out()),
    columns=["Importance"]
).sort_values(by="Importance", ascending=True).T

sns.barplot(
    data=importances.T[::-1],
    x="Importance",
    y=importances.columns[::-1],
    ax=ax
)

## Feature Selection

Before I do feature selection, here is a reusable function to fit the model and return it's scores. It modifies the pipelines to work when some columns are missing

In [ ]:
def fit_with_features(features: list[str]) -> int:
    rfc_pipeline = Pipeline([
        ("transformer", ColumnTransformer([
            ("numeric", Pipeline([
                ("boxcox", boxcox_transformer),
                ("normalizer", normalizing_transformer),
            ]), df[[feature for feature in features if feature != "type"]].columns),
            ("categorical", Pipeline([
                ("onehot", onehot_transformer)
            ]), df[df[["type"] if "type" in features else []].columns].columns)
        ])),
        ("classifier", RandomForestClassifier(
            **bayes_optimal,
            random_state=hash("2100326D") % 2 ** 32,
            n_jobs=-1
        ))
    ])
    
    rfc_pipeline.fit(X_train[features], y_train)

    return accuracy_score(y_test, rfc_pipeline.predict(X_test[features]))

I'm going to run fit the model with all possible combination of the feature list to find which combination gives me the best score for the model.

In [ ]:
running_combinations = True

These values are the optimal values found in the previous run of this notebook.

In [ ]:
combinations_optimal = ['type', 'fixed_acidity', 'volatile_acidity', 'citric_acid', 'residual_sugar', 'free_sulfur_dioxide', 'total_sulfur_dioxide', 'density', 'pH', 'sulphates']
score_optimal = 0

Now depending on whether I've run the combinations already, I will either load the previous data or calculate it again

In [ ]:
mixed_features = numeric_features + categorical_features

if running_combinations:
    count = 0
    total = len([combination for i in range(1, len(mixed_features)) for combination in itertools.combinations(mixed_features, i)])

    for i in range(1, len(mixed_features)):
        for features in itertools.combinations(mixed_features, i):
            features = list(features)

            score = fit_with_features(features)
            count += 1
            print(f"{count} out of {total}")
            print("Accuracy score: %.4f" % score)

            if score > score_optimal:
                print("Highscore!")
                combinations_optimal = features
                score_optimal = score

            print()
    print()
else:
    score_optimal = fit_with_features(combinations_optimal)

print(f"With: {combinations_optimal}")
print(f"Without: {[feature for feature in mixed_features if feature not in combinations_optimal]}")
print("Accuracy score: %.4f" % score_optimal)

In [ ]:
numeric_features = [feature for feature in numeric_features if feature in combinations_optimal]
categorical_features = [feature for feature in categorical_features if feature in combinations_optimal]

X_train = X_train[combinations_optimal]
X_test = X_test[combinations_optimal]

## Final Model

Now that I am done with Feature Engineering, I can build the final version of the model. I will also run bayes again to find the best parameters for my model since my dataset changed a bit.

In [ ]:
running_bayes = True

The parameters here were found through a previous search, in the cases where I don't want to run bayes again since it can be very time consuming during testing

In [ ]:
bayes_optimal = {
    "max_depth": 5036,
    "min_samples_leaf": 1,
    "min_samples_split": 3,
    "n_estimators": 300
}

In [ ]:
rfc_pipeline = Pipeline([
    ("transformer", ColumnTransformer([
        ("numeric", Pipeline([
            ("boxcox", boxcox_transformer),
            ("normalizer", normalizing_transformer),
        ]), df[numeric_features].columns),
        ("categorical", Pipeline([
            ("onehot", onehot_transformer)
        ]), df[categorical_features].columns)
    ])),
    ("classifier", RandomForestClassifier(
        **({} if running_bayes else bayes_optimal),
        random_state=hash("2100326D") % 2 ** 32,
        n_jobs=-1
    ))
])

Now depending on whether I've run BayesSearchCV already, I will either load the best parameters found or calculate them again

In [ ]:
if running_bayes:
    # Fix a bug in scikit-optimize
    np.int = int

    bs = BayesSearchCV(
        rfc_pipeline,
        {
            "classifier__max_depth": (1, 10000),
            "classifier__min_samples_leaf": (1, 5),
            "classifier__min_samples_split": (2, 5),
            "classifier__n_estimators": (1, 1000)
        },
        n_iter=100,
        scoring="accuracy",
        cv=5,
        n_jobs=-1,
        verbose=2
    )

    bs.fit(X_train, y_train)
    bayes_optimal = {
        "max_depth": bs.best_params_["classifier__max_depth"],
        "min_samples_leaf": bs.best_params_["classifier__min_samples_leaf"],
        "min_samples_split": bs.best_params_["classifier__min_samples_split"],
        "n_estimators": bs.best_params_["classifier__n_estimators"]
    }

    print("Best params:", bayes_optimal)
    print("Accuracy Score: %.4f" % accuracy_score(y_test, bs.predict(X_test)))
else:
    rfc_pipeline.fit(X_train, y_train)

    print("Accuracy Score: %.4f" % accuracy_score(y_test, rfc_pipeline.predict(X_test)))

# Exporting the Random Forest Classifier Pipeline

In [ ]:
joblib.dump(rfc_pipeline, "model.pkl")